```{contents}
```
## **Critic Agent in LangGraph**

A **Critic Agent** in LangGraph is a specialized agent whose responsibility is to **evaluate, verify, and improve the output of other agents or nodes**.
It introduces **self-correction, quality control, and reliability** into LLM workflows by acting as an internal reviewer.

---

### **1. Motivation and Role**

LLMs are **generative**, not guaranteed to be correct.
The Critic Agent provides **systematic feedback** to:

| Function           | Purpose                                    |
| ------------------ | ------------------------------------------ |
| Error detection    | Identify incorrect or incomplete reasoning |
| Quality assurance  | Enforce output standards                   |
| Safety enforcement | Catch unsafe or disallowed content         |
| Optimization       | Improve clarity, accuracy, completeness    |
| Verification       | Validate claims and tool results           |

This enables **production-grade robustness**.

---

### **2. Conceptual Position in a LangGraph System**

```
User Input
   ↓
Generator Agent  →  Critic Agent  →  (Revise?) →  Final Output
```

In advanced workflows:

```
Planner → Executor → Critic → Reflect → Revise → Critic → ... → Accept
```

The system forms a **closed feedback loop**.

---

### **3. Critic Agent Design Principles**

| Principle                | Description                                |
| ------------------------ | ------------------------------------------ |
| Separation of roles      | Generator ≠ Critic                         |
| Independent reasoning    | Critic uses different prompt & constraints |
| Deterministic evaluation | Uses checklists, rules, scoring            |
| State-driven             | All feedback stored in shared state        |
| Loop control             | Critic decides whether to continue         |

---

### **4. State Schema Example**

```python
class State(TypedDict):
    task: str
    draft: str
    critique: str
    approved: bool
```

---

### **5. Implementing a Critic Agent in LangGraph**

### **Generator Node**

```python
def generator(state):
    draft = llm.invoke(f"Write solution: {state['task']}")
    return {"draft": draft}
```

### **Critic Node**

```python
def critic(state):
    critique = llm.invoke(f"""
Evaluate this draft for correctness, clarity, and completeness:

{state['draft']}

Respond with:
- Critique
- Verdict: APPROVED or REVISE
""")

    approved = "APPROVED" in critique
    return {"critique": critique, "approved": approved}
```

### **Revision Node**

```python
def revise(state):
    improved = llm.invoke(f"""
Improve the draft using this critique:

Draft:
{state['draft']}

Critique:
{state['critique']}
""")
    return {"draft": improved}
```

---

### **6. Graph Construction**

```python
builder = StateGraph(State)

builder.add_node("generate", generator)
builder.add_node("critic", critic)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "critic")

def decide(state):
    return END if state["approved"] else "revise"

builder.add_conditional_edges("critic", decide, {
    "revise": "revise",
    END: END
})

builder.add_edge("revise", "critic")

graph = builder.compile()
```

This creates a **self-improving feedback loop**.

---

### **7. Common Critic Agent Variants**

| Variant              | Purpose                   |
| -------------------- | ------------------------- |
| Factual Critic       | Verifies correctness      |
| Logical Critic       | Validates reasoning       |
| Safety Critic        | Enforces policies         |
| Style Critic         | Improves tone & structure |
| Domain Expert Critic | Applies expert rules      |
| Scoring Critic       | Produces quality metrics  |

---

### **8. Production Enhancements**

| Feature              | Function                         |
| -------------------- | -------------------------------- |
| Max revision limit   | Prevent infinite loops           |
| Confidence threshold | Exit when quality is high        |
| Human override       | Manual approval                  |
| Critique logging     | Auditing                         |
| Cost control         | Stop early if quality sufficient |

```python
graph.invoke(input, config={"recursion_limit": 5})
```

---

### **9. Why Critic Agents Matter**

| Without Critic     | With Critic           |
| ------------------ | --------------------- |
| Single-pass errors | Iterative improvement |
| Hallucinations     | Reduced hallucination |
| Unstable quality   | Consistent quality    |
| Unsafe outputs     | Policy enforcement    |
| Low trust          | High reliability      |

---

### **10. Mental Model**

> **Generator produces → Critic evaluates → System improves**

This converts LLMs from **creative generators** into **reliable problem solvers** suitable for enterprise systems.



### Demonstration

In [4]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# --- LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Structured Output Schema ---
class CritiqueOutput(BaseModel):
    critique: str = Field(description="Detailed critique of the draft")
    verdict: str = Field(description="Either 'APPROVED' or 'REVISE'")

# --- Shared State ---
class State(TypedDict):
    task: str
    draft: str
    critique: str
    approved: bool
    iteration: int

# --- Generator Agent ---
def generate(state: State):
    draft_response = llm.invoke(f"Write a concise explanation for: {state['task']}")
    return {"draft": draft_response.content, "iteration": 0}

# --- Critic Agent ---
def critic(state: State):
    # Check max iterations safety limit
    MAX_ITERATIONS = 3
    current_iter = state.get("iteration", 0)
    
    if current_iter >= MAX_ITERATIONS:
        print(f"\n--- Max iterations ({MAX_ITERATIONS}) reached, auto-approving ---")
        return {
            "critique": f"Maximum iterations reached. Draft is acceptable.",
            "approved": True
        }
    
    # Use structured output for reliable parsing
    structured_llm = llm.with_structured_output(CritiqueOutput)
    
    critique_result = structured_llm.invoke(f"""
Evaluate the following draft for correctness, clarity, and completeness.

Draft:
{state['draft']}

Provide a critique and verdict. If the draft is accurate and clear, respond with APPROVED. 
If it needs improvement, respond with REVISE and explain what needs to be fixed.
""")
    
    approved = critique_result.verdict.upper() == "APPROVED"
    
    print(f"\n--- Iteration {current_iter} ---")
    print(f"Critique: {critique_result.critique[:150]}...")
    print(f"Verdict: {critique_result.verdict}")
    
    return {"critique": critique_result.critique, "approved": approved}

# --- Revision Agent ---
def revise(state: State):
    improved_response = llm.invoke(f"""
Improve this draft based on the critique. Keep it concise.

Draft:
{state['draft']}

Critique:
{state['critique']}

Provide an improved version:
""")
    return {
        "draft": improved_response.content, 
        "iteration": state.get("iteration", 0) + 1
    }

# --- Graph Construction ---
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("critic", critic)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "critic")

def route(state: State):
    return END if state["approved"] else "revise"

builder.add_conditional_edges("critic", route, {
    "revise": "revise",
    END: END
})

builder.add_edge("revise", "critic")

graph = builder.compile()

# --- Run ---
result = graph.invoke(
    {"task": "What is gradient descent?"},
    config={"recursion_limit": 15}
)

print("\n" + "="*60)
print("FINAL OUTPUT:")
print("="*60)
print(result["draft"])
print(f"\nTotal iterations: {result.get('iteration', 0)}")
print(f"Status: {'Auto-approved (max iterations)' if result.get('iteration', 0) >= 3 else 'Approved by critic'}")


--- Iteration 0 ---
Critique: The draft provides a solid overview of gradient descent, explaining its purpose and basic mechanics. However, there are a few areas that could be impr...
Verdict: REVISE

--- Iteration 1 ---
Critique: The draft provides a solid overview of gradient descent, covering its purpose, mechanism, and some important considerations. However, there are areas ...
Verdict: REVISE

--- Iteration 2 ---
Critique: The draft provides a solid overview of gradient descent, covering its purpose, mechanism, variants, and practical considerations. However, there are a...
Verdict: REVISE

--- Max iterations (3) reached, auto-approving ---

FINAL OUTPUT:
Gradient descent is an optimization algorithm that minimizes a function by iteratively adjusting parameters in the direction of the steepest descent, as indicated by the negative gradient. In machine learning, it is primarily used to minimize the loss function, which quantifies the difference between predicted and actual outcome